# Librerías

In [26]:
import polars as pl
import pandas as pd
import pyarrow

import openai
from openai import OpenAI, RateLimitError, APIError
import kagglehub


import time
start_time_global = time.time()
import json
import os
from dotenv import load_dotenv, find_dotenv 

color = '\033[1m\033[38;5;208m' 
print(f"{color}Versión pandas: {pd.__version__}")
print(f"{color}Versión polars: {pl.__version__}")
print(f"{color}Versión pyarrow: {pyarrow.__version__}")
print(f"{color}Versión OpenAI: {openai.__version__}")

Versión pandas: 2.2.3
Versión polars: 1.27.1
Versión pyarrow: 16.1.0
Versión OpenAI: 1.76.2


In [4]:
#Cargar las variable de entorno
dotenv_path = find_dotenv()
carga_env = load_dotenv(dotenv_path=dotenv_path) 
if carga_env:
    print("Archivo .env encontrado y cargado exitosamente.")
else:
    print("No se encontró el archivo .env o no se pudo cargar.")

Archivo .env encontrado y cargado exitosamente.


# Categorización de las reseñas por DeepSeek

A continuación, enviamos cada una de las reseñas seleccionados del dataset de Yelp para la categorización pora el modelo de IA DeepSeek.

In [5]:
# --- 1. Configuración de la API Key y Base URL de DeepSeek ---
try:
    deepseek_api_key = os.environ["DEEPSEEK_API_KEY"]
    deepseek_base_url = os.environ["DEEPSEEK_API_URL"]
    deepseek_time_out = os.environ['DEEPSEEK_TIMEOUT']
    print("Clave API y URL base de DeepSeek configuradas correctamente.")
except KeyError:
    print("Error: Las variables de entorno 'DEEPSEEK_API_KEY' o 'DEEPSEEK_BASE_URL' no están configuradas.")

Clave API y URL base de DeepSeek configuradas correctamente.


A continuación la creación del comportamiento del prompt con la técnica de few-shot prompting.

In [23]:
examples_data_genuine_fake = [
    {
        "text": "Many comments say it's the best on the market, but I don't think it's that good. Ultimately, I think it's the name or the fame. For me, it's not worth investing the money in this product. That's just my opinion. Some might say it's the least of it, but not for me.",
        "stars": 5,
        "stars_business": 4.1,
        "review_count_business": 394,
        "average_stars_user": 4,
        "review_count_user": 35,
        "label": "Genuine"
    },
    {
        "text": "Had a wonderful experience. The product arrived on time and exactly as described. The quality is excellent for the price.",
        "stars": 5,
        "stars_business": 4.6,
        "review_count_business": 1200,
        "average_stars_user": 4,
        "review_count_user": 18,
        "label": "Genuine"
    },
    {
        "text": "Brilliant wheel cleaner removes baked on brake dust .",
        "stars": 5,
        "stars_business": 4.4,
        "review_count_business": 1142,
        "average_stars_user": 1,
        "review_count_user": 16,
        "label": "Genuine"
    },
    {
        "text": "THE BEST PLACE IN THE CITY!!! INCREDIBLE!!! You have to go NOW! 5 STARS!!!",
        "stars": 5,
        "stars_business": 3.2, 
        "review_count_business": 45,  
        "average_stars_user": 5, 
        "review_count_user": 1,     
        "label": "Fake"
    },
    {
        "text": "I didn't like it at all. Everything was bad. Horrible. I wouldn't recommend it whatsoever. Very poor.",
        "stars": 1,
        "stars_business": 4.5, 
        "review_count_business": 950,  
        "average_stars_user": 1, 
        "review_count_user": 1,     
        "label": "Fake"
    },
    {
        "text": "Apart from a simonizer, it doesn't hide scratches! still visible!",
        "stars": 1,
        "stars_business": 3, 
        "review_count_business": 21,  
        "average_stars_user": 1, 
        "review_count_user": 1,     
        "label": "Fake"
    }
]

Seguimos con el formateo de los ejemplos en una unica cadena de texto.

In [24]:
# Creación de la estructura de los ejemplos 
formatted_examples_with_metadata = ""
for ex in examples_data_genuine_fake:
    formatted_examples_with_metadata += f"""Example Review Text: {ex['text']}
        Example Review Stars: {ex['stars']}
        Example Business Avg Stars: {ex['stars_business']}
        Example Business Review Count: {ex['review_count_business']}
        Example User Avg Stars: {ex['average_stars_user']}
        Example User Review Count: {ex['review_count_user']}
        Category: {ex['label']}
        ---
        """
# Eliminación del ultimo '---' para limpiar el texto del prompt
if formatted_examples_with_metadata.endswith("---\n"):
    formatted_examples_with_metadata = formatted_examples_with_metadata[:-4]

A contiuación, tenemos la definición base del prompt.

In [25]:
classification_prompt_template = f"""
* Task: Classify the customer review provided at the end into a single category: Genuine or Fake, considering the review text AND associated metadata.

* Metadata Provided:
- Review Stars: Rating (1-5) in this specific review.
- Business Avg Stars: The business's average rating across all reviews.
- Business Review Count: Total number of reviews for the business.
- User Avg Stars: The average rating given by this user across all their reviews.
- User Review Count: Total number of reviews written by this user.

* Category Definitions:
- Genuine: The review appears to be from a real customer with an authentic experience. Details in text are often plausible and consistent with metadata patterns (e.g., user has some review history, rating aligns somewhat with user/business averages unless specific reasons given in text).
- Fake: The review appears artificial, possibly promotional or defamatory. May show metadata inconsistencies (e.g., user with 1 review count + extreme generic rating, rating wildly different from business average without clear textual explanation). Text details (or lack thereof) are primary, but metadata provides important context.

* Examples of Classification (including metadata):
--- START EXAMPLES ---
{formatted_examples_with_metadata}
--- END EXAMPLES ---

* Now, classify the following review using its text and metadata:

Review Text: {{review_text}}
Review Stars: {{stars}}
Business Avg Stars: {{stars_business}}
Business Review Count: {{review_count_business}}
User Avg Stars: {{average_stars_user}}
User Review Count: {{review_count_user}}

* Output Instructions: Return ONLY the category name ('Genuine' or 'Fake') and nothing else.

Category:"""

A continuación una función para controlar la ultima ejecución del dataset y no iniciar desde el principio

In [16]:
# Lógica para reanudar el procesamiento
# Función para leer índices procesados
def get_processed_indices(temp_file_path: str) -> set[int]:
    """
    Lee el archivo temporal JSON Lines y devuelve un conjunto de índices
    de reseñas que ya han sido procesadas.

    Args:
        temp_file_path: La ruta al archivo temporal JSON Lines.

    Returns:
        Un conjunto de enteros representando los índices procesados.
    """
    processed_indices = set()
    if os.path.exists(temp_file_path):
        print(f"\nArchivo temporal '{temp_file_path}' encontrado. Leyendo índices procesados...")
        try:
            with open(temp_file_path, 'r', encoding='utf8') as f:
                for line_num, line in enumerate(f):
                    line = line.strip()
                    if not line: # Saltar líneas vacías
                        continue
                    try:
                        data = json.loads(line)
                        # Asumimos que el índice se guarda bajo la clave 'index'
                        if 'index' in data and isinstance(data['index'], int):
                            processed_indices.add(data['index'])
                        else:
                             print(f"Advertencia: Línea {line_num + 1} en '{temp_file_path}' no contiene un índice válido.")
                    except json.JSONDecodeError:
                        print(f"Advertencia: Ignorando línea no válida/incompleta en el archivo temporal (línea {line_num + 1}).")
                        # Si la decodificación falla, asumimos que es una línea parcial al final
                        break # Detener la lectura aquí
            print(f"Encontrados {len(processed_indices)} reseñas procesadas anteriormente.")
        except Exception as e:
            print(f"Error al leer el archivo temporal '{temp_file_path}': {e}")
            print("Se procesarán todas las reseñas desde el inicio por precaución.")
            processed_indices = set() # Si hay un error de lectura, empezar de cero
    else:
        print(f"\nArchivo temporal '{temp_file_path}' no encontrado. Iniciando procesamiento desde el inicio.")

    return processed_indices


A continuación inicia la conexión con DeepSeek para la categorización de las reseñas.

In [21]:
start_time=time.time()
explicit_work_path = os.getenv('HOME_WORK')
deepseek_model_name = "deepseek-chat"
print('Inicia procesamiento de los datos en DeepSeek')
# --- Configurar el cliente de OpenAI para usar el endpoint de DeepSeek ---
client = OpenAI(
    base_url=deepseek_base_url,
    api_key=deepseek_api_key,
    timeout=deepseek_time_out
)
# modelo de DeepSeek para tareas
# Lectura del fichero de las reseñas seleccionadas de yelp
review_selected_file_path = f'{explicit_work_path}yelp_academic_dataset_review_selected.jsonl'
df_data_reviews_selected_pl = pl.read_json(review_selected_file_path)
print(f"\nTotal de reseñas a procesar: {df_data_reviews_selected_pl.shape[0]}")

# Archivos de salida de las reseñas categorizadas
reviews_categorize_output_file = f'{explicit_work_path}final_reviews_categorize.jsonl'

# Obtener los índices ya procesados llamando a la nueva función
processed_indices = get_processed_indices(reviews_categorize_output_file)

Inicia procesamiento de los datos en DeepSeek

Total de reseñas a procesar: 17000

Archivo temporal '/home/teneo/fakereviews/final_reviews_categorize.jsonl' no encontrado. Iniciando procesamiento desde el inicio.


In [ ]:
# Abrir el archivo temporal
with open(reviews_categorize_output_file, 'a', encoding='utf8') as reviews_file:
    # Iterar sobre las filas del DataFrame de Polars
    print(f"\nIniciando/Reanudando procesamiento")
    for row_index,review_row_text in enumerate(df_data_reviews_selected_pl.head(17000).select(['text']).iter_rows()):
        
        # --- Control de reanudación: Saltar si este índice ya fue procesado ---
        if row_index in processed_indices:
            #print(f"Saltando reseña con Índice {row_index}, ya procesada.")
            continue # Ir a la siguiente iteración del bucle

        # Esta reseña NO ha sido procesada
        review_text = review_row_text[0]
        current_identifier = row_index

        full_prompt_for_review = classification_prompt_template + review_text
        classification_result = "Error al procesar"
        processing_status = "Failed"
        error_message = ""

        #print(f"Procesando Índice: {current_identifier}.")

        try:
            api_time_start = time.time()
            # Conexión al API
            response = client.chat.completions.create(
                model=deepseek_model_name,
                messages=[
                    {"role": "user", "content": full_prompt_for_review}
                ],
                temperature=1.3,
                max_tokens=20
            )

            classification_result = response.choices[0].message.content.strip()
            processing_status = "Success"
            print(f"Índice: {current_identifier}, Clasificación: {classification_result}, Tiempo: {time.time() - api_time_start:.2f}")

        except Exception as e:
            error_message = str(e)
            print(f"Error al procesar la reseña con Índice {current_identifier}: {error_message}")
            classification_result = "Error: " + error_message[:50]
            processing_status = "Failed"
            print(e)
            import traceback
            traceback.print_exc()

        # Preparar datos para guardar y escribir en el archivo
        result_data = {
            "index": current_identifier, # Guardamos el índice de la fila
            "review_text": review_text,
            "classification": classification_result,
            "processing_status": processing_status,
            "error_message": error_message
        }

        try:
            json_line = json.dumps(result_data, ensure_ascii=False)
            reviews_file.write(json_line + '\n')
            # Escribir en disco para no perder los datos en caso de fallo
            reviews_file.flush() 

        except Exception as write_error:
            print(f"ERROR FATAL: No se pudo escribir el resultado temporal para el Índice {current_identifier}: {write_error}")
            # Aquí podrías considerar detener el script sys.exit(1)

print(f"\nFinalizado el bucle de procesamiento.")




print(f'Total tiempo ejecución: {time.time() - start_time:.2f}')

Inicia procesamiento de los datos en DeepSeek

Total de reseñas a procesar: 239112

Archivo temporal '/home/aam/fakereviews/final_reviews_categorize.jsonl' encontrado. Leyendo índices procesados...
Encontrados 1000 reseñas procesadas anteriormente.

Iniciando/Reanudando procesamiento
Índice: 1000, Clasificación: Genuine negative, Tiempo: 5.02
Índice: 1001, Clasificación: Genuine positive, Tiempo: 5.89
Índice: 1002, Clasificación: Genuine negative, Tiempo: 5.82
Índice: 1003, Clasificación: Genuine positive, Tiempo: 4.28
Índice: 1004, Clasificación: Genuine positive, Tiempo: 5.72
Índice: 1005, Clasificación: Genuine positive, Tiempo: 6.39
Índice: 1006, Clasificación: Genuine positive, Tiempo: 4.65
Índice: 1007, Clasificación: Genuine positive, Tiempo: 6.45
Índice: 1008, Clasificación: Genuine negative, Tiempo: 4.55
Índice: 1009, Clasificación: Genuine positive, Tiempo: 5.24
Índice: 1010, Clasificación: Genuine positive, Tiempo: 4.32
Índice: 1011, Clasificación: Genuine positive, Tiempo: 